In [1]:
### 0. import
import psycopg2
import numpy  as np
import pandas as pd
from datetime import datetime

# display option (float 자리수)
pd.options.display.float_format = lambda x: f'{x:.4f}'


### 1. 데이터 가져오기
# AWS DW Connect
aws_conn = psycopg2.connect(
    host = 'redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
    port = 5439,
    dbname = 'dev',
    user = 'awsuser',
    password = 'cremaoAdmin1234qwer!!'
)
aws_cursor = aws_conn.cursor()

In [2]:
# 인스타그램 게시글 수 가져오기
sql = 'select * from m1.instagram_emd_cnt'
aws_cursor.execute(sql)
aws_conn.commit()
rows = aws_cursor.fetchall()

df_instagram = pd.DataFrame(rows)
df_instagram.columns = ['emd_nm','emd_cd','cnt','base_ym']

In [3]:
df_instagram

,emd_nm,emd_cd,cnt,base_ym
0,가남읍,4167025000,5597,202210
1,가대동,3120010400,4728,202210
2,가락동,1171010700,4146,202210
3,가례면,4872031000,2385,202210
4,가북면,4888041000,2731,202210
...,...,...,...,...
3963,흑산면,4691036000,3135,202210
3964,흑석동,2920012400,43771,202210
3965,흑암동,4518012300,48,202210
3966,흥각동,4725011700,2015,202210


In [4]:
# 법정동별 유동인구 
sql = '''
    select substring (pnu, 0, 11), sum(f_tot)
      from m2.cremao_land_floating_population
     group by substring (pnu, 0, 11)
'''
aws_cursor.execute(sql)
aws_conn.commit()
rows = aws_cursor.fetchall()

df_f_tot = pd.DataFrame(rows)
df_f_tot.columns = ['emd_cd','f_tot']

In [5]:
df_f_tot

,emd_cd,f_tot
0,1111010100,5717165
1,1111010200,4621997
2,1111010300,1106859
3,1111010600,39911228
4,1111011000,51802938
...,...,...
18597,4889039030,152
18598,4889041026,535
18599,4889041030,777
18600,4889042022,126


In [6]:
df = pd.merge(df_instagram, df_f_tot, on='emd_cd', how='left')

In [7]:
DF = df[~df['f_tot'].isnull()].reset_index(drop=True)

In [13]:
# inf 값 제거
DF = DF.drop([249,116])

In [8]:
# 계산할 컬럼 타입 변경
DF['cnt'] = DF['cnt'].astype(float)

In [14]:
# 비율 계산식
DF['calculate'] = (DF['f_tot'] / DF['cnt']) * 100

In [16]:
DF

,emd_nm,emd_cd,cnt,base_ym,f_tot,calculate
0,가대동,3120010400,4728.0000,202210,7799.0000,164.9535
1,가락동,1171010700,4146.0000,202210,504343057.0000,12164569.6334
2,가수동,4137012100,2514.0000,202210,1499240.0000,59635.6404
3,가수원동,3017011400,13165.0000,202210,66768338.0000,507165.4994
4,가야동,2623011000,36416.0000,202210,285267295.0000,783357.0271
...,...,...,...,...,...,...
2737,휘경동,1123010900,26827.0000,202210,266814686.0000,994575.1892
2738,휴천동,4721010400,34591.0000,202210,164093492.0000,474382.0416
2739,흑석동,2920012400,43771.0000,202210,34090127.0000,77882.9065
2740,흑암동,4518012300,48.0000,202210,23582.0000,49129.1667


In [17]:
DF['calculate'].describe()

count         2740.0000
mean      16059786.8885
std      174342139.3778
min              0.1772
25%          14908.8882
50%         172472.5496
75%        1077322.3977
max     5694416058.5366
Name: calculate, dtype: float64

In [20]:
# 50% 보다 높으면 1, 낮으면 0
DF.sort_values('calculate',ascending=True)

,emd_nm,emd_cd,cnt,base_ym,f_tot,calculate
112,구완동,3014012100,8463.0000,202210,15.0000,0.1772
993,이기동,4217012500,5149.0000,202210,16.0000,0.3107
2236,청소면,4418034000,240164.0000,202210,1200.0000,0.4997
2538,지정동,2920015200,94522.0000,202210,696.0000,0.7363
1171,금고동,3020014800,72707.0000,202210,573.0000,0.7881
...,...,...,...,...,...,...
923,우동,2635010500,38.0000,202210,680687737.0000,1791283518.4211
746,신월동,1147010300,36.0000,202210,726230988.0000,2017308300.0000
1150,구로동,1153010200,56.0000,202210,1437855067.0000,2567598333.9286
577,상계동,1135010500,27.0000,202210,1333539468.0000,4939035066.6667


In [ ]:
### 3. DB INSERT
import datetime

table = 'm2.datadam_f_tot_per_sns'

# execute_mogrify
def execute_mogrify(conn, df, table, val):
    # Create a list of tuples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL query to execute
    cursor = conn.cursor()
    values = [cursor.mogrify(val, tup).decode('utf8') for tup in
              tuples]
    query = "INSERT INTO %s(%s) VALUES " % (table, cols) + ",".join(values)

    try:
        cursor.execute(query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    cursor.close()

# values 설정
val = '(' + ('%s,' * len(df.columns))[:-1] + ')'

# insert
for j in range(0, len(df), 10000):

    # DB Connect
    conn = psycopg2.connect(
        host='redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
        port=5439,
        dbname='dev',
        user='awsuser',
        password='cremaoAdmin1234qwer!!'
    )

    # 10000개 단위로 나누기
    tmp = df[j:j + 10000]

    # insert
    execute_mogrify(conn, tmp, table, val)
    print(datetime.datetime.now(), ' : ', j)
    tmp = pd.DataFrame()

# row count
print(len(df))